Before we begin this is my explanation of the model we are about to deploy.
BERT model is known to be amazing for pretraining, as it is also sort of required. This is because the model is quite massive with 110M parameters when we are talking about BERTBase. Now this model was specifically meant for sentiment analysis which likely means it was fine tuned to fit sentiment type of data.
How does BERT work? Here is my best explanation possible.
1. Every single input we give it is labeled with 3 different things. First thing: token embedding, this token embedding is made from a word embedding algorithm (I am familiar with Word2Vec and its variations such as skipgram and cbow!). It is also given a positional encoding, with the sinusoidal function(This can also be learned but it was shown that they both result in similar performance in the paper).
It is lastly given a segment token, which describes which sequence we are currently in, within the input. For example if we give the input two sentences, and we are looking at a word in the first sentence its associated segment embedding will be 1, which defines sentence 1, or 0 if its 0 based i dont really know lol. 
2.We then input these into masked multihead self attention(i believe 12 multiheads were used in paper for pretraining), and then the way these self attention mechanisms work is the following. Lets say we are are index 2, and we are performing self attention, what we do is compute the dot product between index 2's query and every other inputs key value, to get a returned output, we softmax this output(in the paper we also perform normalization by the dimension of the head we are working with).
We then multiply each of these results with each their respective values. Then we concatenate this as the representation for index 2! We do this from index 0 to index n where n represents the length of the input. 
So this is how regular self attention works, in multihead self attention what we end up doing is splitting the input into segments, but still go over all n inputs when doing self attention. What i mean by this is that for index 0 lets say, when we do multihead self attention, we perform attention with the first split, then the second split and so on until we hit the number of splits we decided we wanted with self attention. We concatenate these results, this lets us see which of the inputs we are most tied to in the case where a specific word can have multiple meanings due to ambiguous context. The masked portion of this means that for 15% of the inputs, we are 'masking' them which means they are the stuff we are trying to predict as they are removed from our input. This is the really genius part of this model, as the problem associated with bidirectionality predicting (being able to look into the future) is now solved. The model only predicts the missing tokens(masked). Specifically, in the paper it states that masking is 15% of total input, but for the 15% that is masked, 80% is actually masked, 10% is a random word, and the last 10% is the actual word itself. Note we are still predicting the word even in the case where it is the actual word itself. Im sure the model keeps track of the indices to do this. 
3. After the Multihead masked self attention is done, we then feed it into a layer normalization, which is simply a function that looks at the neural network layer we are in, takes the mean of all the values, takes the variance, then subtracts each specific value by the mean and divides that entire thing by the square root of the variance plus some e constant, to avoid division by 0. A long with the layernormalization we also have a residual connection which helps with training/backpropogating to farther inputs quicker. 
4. We then feed into a feedforward network, each of the inputs have their own specific neuron in this ffn. 
5. Another LayerNormalization + Residual Layer is connected after this FFN.
6. We stack 12 of these ENCODERS, in order to make BERTbase, and 24 
to make BERTlarge. This is then pretrained of course on tons of data and then fine tuned to our specific task!(downstream task).


Last thing I wanted to say about bert is that the beauty comes from the bidirectionality that comes with encoders, but the solution came from the masking portion.

1.Install and import Dependencies
2. Instantiate model
3. Encode and calculate sentiment
4.Collect reviews
5.Load reviews into dataframe and score


In [ ]:
#!pip install transformers requests beautifulsoup4 pandas numpy
#Go to pytorch website to find command to deploy dependencies

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
import torch
import requests
from bs4 import BeautifulSoup
import re
#Tokenizer is for words/characters to parse strings to a sequence of numbers
#AutoModelForSequenceClassification gives us transformer architecture
#We just use argmax function from torch
#Requests gets data from yelp
#bs4 import beautifulsoup lets us traverse dom results from yelp
#re lets us get regex function to get functions we want

Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#This tokenizer and model are specific to the pretrained BERT base multilingual model!

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

3. Encode and Calculate Sentiment


In [ ]:
tokens = tokenizer.encode("It was good but could've been great!", return_tensors = 'pt')
#'pt' is pytorch

In [ ]:
tokens

tensor([[  101, 10197, 10140, 12050, 10502, 12296,   112, 10312, 10662, 11838,
           106,   102]])

In [ ]:
tokenizer.decode(tokens[0])

"[CLS] it was good but could've been great! [SEP]"

In [ ]:
#We have to pass tokens to model to perform sentiment analysis


In [ ]:
result = model(tokens)

In [ ]:
result

SequenceClassifierOutput([('logits',
                           tensor([[-1.5436,  0.3746,  1.9667,  0.9602, -1.5450]],
                                  grad_fn=<AddmmBackward0>))])

The position of the highest value in the output is what defines the class that the model put the sentence in!

In [ ]:
torch.argmax(result[0])

tensor(2)

In [ ]:
#The above shows us that the class it was classified as was 0 which is unhappy! Which is accurate


In [ ]:
int(torch.argmax(result[0]))

2

4. Collect Reviews From Yelp!

In [ ]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [t.text for t in results]
reviews

['The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 "Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someo

In [ ]:
#r.text This is everything from the web page
#BeatifulSoup takes this in and then since this is html code it uses the html parser
#then regex specifies what kind of class label we are looking for
#Then results takes this and uses soup to find all instances of comment in 'class'
#The 'p' defines that we are looking for paragraphs that are under the class comment!
#We can find these things by simply inspecting the web page !
#Then result.text for result in results uses 'result' and grabs the '.text' portion
#This portion has all of the parts of the content that is not html, rather just the users review


5. Load Reviews Into DataFrame and then Score them!

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns =['reviews'])
#Making sure to convert to numpy array because thats whats compatible with pandas dataframe

In [ ]:
df

,reviews
0,The food is fresh and tasty. The scallop cevi...
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,We came here on a Thursday night @ 5pm and by ...
4,I was pleasantly surprised at what a great job...
5,Really nice (upmarket) Mexican restaurant. Goo...
6,Ordered feed me for $59 along with that.. Food...
7,"The best tasting cauliflower taco, mushroom ce..."
8,Have been here twice and have absolutely loved...
9,If you're looking for a quiet little romantic ...


In [ ]:
#We are going to make a function that does the sentiment scoring for us

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors = 'pt')
  result = model(tokens)
  return int(torch.argmax(result[0]))+1 

In [ ]:
sentiment_score(df['reviews'].iloc[0])

4

In [ ]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df['sentiment'].mean() #Average review is 3.9 stars!

3.9

In [ ]:
#We can apply this to any website that has a review section that labels its reviews under
#a p tag and has the class 'comment' associated with each one of their reviews. We can easily
#check this by clicking on the review, clicking inspect element on it, and then looking at its
#parent tag as well as its parent class. This will specify where exactly we need to search for
#our data!